In [1]:
# Import required libraries
import ssl
import urllib.request
import datetime as dt
import certifi

# Define the URL to fetch GPS TLE data
tle_url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=gps-ops&FORMAT=tle'
tle_file = 'gps-ops.txt'  # Name of the local file to save data

# Create an SSL context using certifi to avoid SSL errors
context = ssl.create_default_context(cafile=certifi.where())

# Open the URL and save the TLE data manually
with urllib.request.urlopen(tle_url, context=context) as response, open(tle_file, 'wb') as out_file:
    out_file.write(response.read())  # Save the downloaded data to a file

# Open the downloaded file and read the first satellite's TLE data
with open(tle_file, 'rt') as tlefile:
    line0 = tlefile.readline().strip()  # Satellite name
    line1 = tlefile.readline().strip()  # First line of TLE
    line2 = tlefile.readline().strip()  # Second line of TLE

# Print the first satellite's TLE set
print("Satellite Name:", line0)
print("TLE Line 1:", line1)
print("TLE Line 2:", line2)

Satellite Name: GPS BIIR-2  (PRN 13)
TLE Line 1: 1 24876U 97035A   25076.50113901  .00000022  00000+0  00000+0 0  9993
TLE Line 2: 2 24876  55.7681 116.4396 0087517  53.8685 307.0158  2.00562662202797


In [2]:
# Extract the Epoch Time from TLE (Characters 18-32 from Line 1)
refepochString = line1[18:32]  # Example: '24057.09840884'
print("\nExtracted Epoch String:", refepochString)

# Split the string into two parts: Year-Day and Fractional Day
RefEpochDate = dt.datetime.strptime(refepochString[:5], '%y%j')  # Convert '24057' to datetime
RefEpochTime = dt.timedelta(days=float(refepochString[5:]))  # Convert fractional day to timedelta

# Compute the full date-time of the TLE Epoch
RefEpochdt = (RefEpochDate + RefEpochTime).replace(tzinfo=dt.timezone.utc)

# Print results
print("\nReference Epoch Date:", RefEpochDate)
print("Fractional Time (as timedelta):", RefEpochTime)
print("Final Computed Epoch Datetime:", RefEpochdt)


Extracted Epoch String: 25076.50113901

Reference Epoch Date: 2025-03-17 00:00:00
Fractional Time (as timedelta): 12:01:38.410464
Final Computed Epoch Datetime: 2025-03-17 12:01:38.410464+00:00


In [3]:
# Format the TLE Epoch time into a readable format
formattedEpoch = RefEpochdt.strftime('%d %b %Y %H:%M:%S.%f')
print("\nFormatted TLE Epoch Time:", formattedEpoch)

# Extract YYYYDDD format
YYddd = int(RefEpochdt.strftime('%y%j'))
print("\nTLE Epoch in YYDDD format:", YYddd)

# Convert a future date string into a datetime object
StartString = '02 May 2024 14:00:00.00'
Startdt = dt.datetime.strptime(StartString, '%d %b %Y %H:%M:%S.%f')
print("\nConverted Start Time:", Startdt)

Startdt = Startdt.replace(tzinfo=dt.timezone.utc)
print("\nConverted Start Time:", Startdt)

# Compute the number of seconds from TLE Epoch to Start Time
Epseconds = (Startdt - RefEpochdt).total_seconds()
print("\nSeconds from TLE Epoch to Start Time:", Epseconds)


Formatted TLE Epoch Time: 17 Mar 2025 12:01:38.410464

TLE Epoch in YYDDD format: 25076

Converted Start Time: 2024-05-02 14:00:00

Converted Start Time: 2024-05-02 14:00:00+00:00

Seconds from TLE Epoch to Start Time: -27554498.410464


In [4]:
# Computing Time Delta from TLE Epoch to Tracking Time
trackingtime = dt.datetime.now(dt.timezone.utc)
print("\nCurrent UTC Tracking Time:", trackingtime)

# Adjust tracking time to a specific UTC hour (e.g., 17:00 UTC)
trackingtime = trackingtime.replace(hour=17, minute=0, second=0, microsecond=0)
print("Adjusted UTC Tracking Time:", trackingtime)

# Compute the time delta between TLE Epoch and Tracking Time
deltat = trackingtime - RefEpochdt
print("\nTime Delta (Tracking Time - TLE Epoch):", deltat)

# Convert the time delta into seconds
tstart = deltat.total_seconds()
print("\nSeconds from TLE Epoch to Tracking Time:", tstart)



Current UTC Tracking Time: 2025-03-18 00:25:28.531373+00:00
Adjusted UTC Tracking Time: 2025-03-18 17:00:00+00:00

Time Delta (Tracking Time - TLE Epoch): 1 day, 4:58:21.589536

Seconds from TLE Epoch to Tracking Time: 104301.589536
